In [99]:
import math
import numpy as np
import scipy.signal
import pandas as pd


In [3]:
def compute_air_absorption_coefficients(T, p, rel_hum, freqs):
        """
        Computes air absorption coefficients at a set of nbands equispaced frequencies in the range `[0, fs]`, based
        on the ISO 9613-1 standard. 

        A different formulation can be found in 
        `Keith Attenborough, "Sound Propagation in the Atmosphere", Springer Handbook of Acoustics`, and can be set
        by using T01 = 293.15
        The coefficients depend on the atmsopsheric temperature, pressure and relative humidity.

        Parameters
        ----------
        nbands: int
            Number of frequency bands in which to compute air absorption coefficients, by default 50

        Returns
        -------
        np.ndarray
            1D Array containing nbands air absorption coefficients, computed in dB scale, at the defined 
            set of frequencies

        """

        T0 = 293.15     # Standard room temperature T = 20deg Celsius
        T01 = 273.16    # Triple point isotherm temperature, ISO 9613-1
        T = T + 273.15
        ps0 = 1         # Standard atmospheric pressure in atm

        f = freqs # np.linspace(0, fs, num=nbands)     # Frequencies in which coeffs will be computed
        
        Csat = -6.8346 * math.pow(T01 / T, 1.261) + 4.6151
        rhosat = math.pow(10, Csat)
        H = rhosat * rel_hum * ps0 / p

        frn = (p / ps0) * math.pow(T0 / T, 0.5) * (
                9 + 280 * H * math.exp(-4.17 * (math.pow(T0 / T, 1/3.) - 1)))

        fro = (p / ps0) * (24.0 + 4.04e4 * H * (0.02 + H) / (0.391 + H))

        alpha = f * f * (
            1.84e-11 / ( math.pow(T0 / T, 0.5) * p / ps0 )
            + math.pow(T / T0, -2.5)
            * (
                0.10680 * math.exp(-3352 / T) * frn / (f * f + frn * frn)
                + 0.01278 * math.exp(-2239.1 / T) * fro / (f * f + fro * fro)
                )
            )
        
        return alpha * 20 / np.log(10)

def compute_air_absorption_filter(alpha, freqs, distance, numtaps):
        """
        Computes air absorption filter as a FIR filter with `numtaps` coefficients. The filter depends
        on the distance travelled by the sound wave between the source and the receiver.

        Parameters
        ----------
        distance : float
            Distance travelled by the sound wave, in meters
        numtaps : int
            Number of coefficients of the FIR filter

        Returns
        -------
        np.ndarray
            1D array containing `numtaps` FIR filter coefficients modelling air absorption
        """

        alpha_lin = 10 ** (-alpha * distance / 20)     # Convert coeffs in dB to linear scale
        freqs_norm = freqs / max(freqs)
        filt_coeffs = scipy.signal.firwin2(numtaps, freqs_norm, alpha_lin)
        
        return filt_coeffs

In [62]:
# Parameters
T = 20
p = 1
rel_hum = 50
fs = 8000
numbands = 8
freqs = np.linspace(0, fs, numbands)
alpha = compute_air_absorption_coefficients(T, p, rel_hum, freqs)

# Distance
d = 10

# Filter
coeffs = compute_air_absorption_filter(alpha, freqs, d, 11)
np.reshape(coeffs, (1,11))

array([[ 8.38634575e-05, -1.85702754e-04,  1.02496287e-03,
        -3.37783948e-03,  2.11923385e-02,  9.57727940e-01,
         2.11923385e-02, -3.37783948e-03,  1.02496287e-03,
        -1.85702754e-04,  8.38634575e-05]])

In [84]:
# Create dataset
T = np.arange(-1,1)       # Range of temperatures
p = 1
rel_hum = np.arange(0,10,5)
d = np.arange(0,2,0.5)

# alpha = np.zeros((len(T), len(rel_hum), numbands))
coeffs = np.empty((0, 11))
abs_coeffs = np.empty((0, numbands))
for i in range(len(T)):
    for j in range(len(rel_hum)):
        alpha = compute_air_absorption_coefficients(T[i],p, rel_hum[j], freqs)
        abs_coeffs = np.append(abs_coeffs, np.tile(np.reshape(alpha, (1,numbands)), (len(d),1)), axis = 0)
        for k in range(len(d)):
            coeffs = np.append(coeffs, np.reshape(compute_air_absorption_filter(alpha, freqs, d[k], numtaps=11), (1,11)), axis = 0)

print(np.shape(abs_coeffs))
print(abs_coeffs)


(16, 8)
[[0.         0.00110474 0.00170841 0.00271411 0.00412204 0.00593221
  0.00814463 0.01075932]
 [0.         0.00110474 0.00170841 0.00271411 0.00412204 0.00593221
  0.00814463 0.01075932]
 [0.         0.00110474 0.00170841 0.00271411 0.00412204 0.00593221
  0.00814463 0.01075932]
 [0.         0.00110474 0.00170841 0.00271411 0.00412204 0.00593221
  0.00814463 0.01075932]
 [0.         0.00567815 0.00635442 0.00737378 0.0087865  0.01059889
  0.01281253 0.01542794]
 [0.         0.00567815 0.00635442 0.00737378 0.0087865  0.01059889
  0.01281253 0.01542794]
 [0.         0.00567815 0.00635442 0.00737378 0.0087865  0.01059889
  0.01281253 0.01542794]
 [0.         0.00567815 0.00635442 0.00737378 0.0087865  0.01059889
  0.01281253 0.01542794]
 [0.         0.00112487 0.00172966 0.0027372  0.00414771 0.00596121
  0.00817769 0.01079718]
 [0.         0.00112487 0.00172966 0.0027372  0.00414771 0.00596121
  0.00817769 0.01079718]
 [0.         0.00112487 0.00172966 0.0027372  0.00414771 0.005

In [69]:
coeffs = compute_air_absorption_filter(alpha, freqs, d, 11)
coeffs_exp = np.tile(np.reshape(coeffs, (1,11)), (10,1))
test = np.empty((0,11), float)
test = np.append(test, coeffs_exp, axis = 0)
print(test)


[[ 8.38634575e-05 -1.85702754e-04  1.02496287e-03 -3.37783948e-03
   2.11923385e-02  9.57727940e-01  2.11923385e-02 -3.37783948e-03
   1.02496287e-03 -1.85702754e-04  8.38634575e-05]
 [ 8.38634575e-05 -1.85702754e-04  1.02496287e-03 -3.37783948e-03
   2.11923385e-02  9.57727940e-01  2.11923385e-02 -3.37783948e-03
   1.02496287e-03 -1.85702754e-04  8.38634575e-05]
 [ 8.38634575e-05 -1.85702754e-04  1.02496287e-03 -3.37783948e-03
   2.11923385e-02  9.57727940e-01  2.11923385e-02 -3.37783948e-03
   1.02496287e-03 -1.85702754e-04  8.38634575e-05]
 [ 8.38634575e-05 -1.85702754e-04  1.02496287e-03 -3.37783948e-03
   2.11923385e-02  9.57727940e-01  2.11923385e-02 -3.37783948e-03
   1.02496287e-03 -1.85702754e-04  8.38634575e-05]
 [ 8.38634575e-05 -1.85702754e-04  1.02496287e-03 -3.37783948e-03
   2.11923385e-02  9.57727940e-01  2.11923385e-02 -3.37783948e-03
   1.02496287e-03 -1.85702754e-04  8.38634575e-05]
 [ 8.38634575e-05 -1.85702754e-04  1.02496287e-03 -3.37783948e-03
   2.11923385e-02  

In [79]:
counter = 0
for i in range(len(T)):
    for j in range(len(rel_hum)):
        counter +=1
        print(counter)

1
2
3
4


In [98]:
id = 'T=' + str(T[0]) + 'rel_hum=' + str(rel_hum[0]) + 'd='+str(d[0])
print(id)

alpha = abs_coeffs[0]
print(alpha)
print(d[0])
print(coeffs[0])

T=-1rel_hum=0d=0.0
[0.         0.00110474 0.00170841 0.00271411 0.00412204 0.00593221
 0.00814463 0.01075932]
0.0
[ 8.05336877e-18 -3.49412546e-17  6.80497046e-17 -6.79669532e-17
  2.11940188e-16  1.00000000e+00 -2.08775839e-16  6.48256990e-17
 -7.45165152e-17  3.49412546e-17 -8.27962252e-18]


In [110]:
data = {'id': id, 'alpha': list(alpha), 'distance': d[0], 'absorption_filter': coeffs[0]}
# df = pd.DataFrame(data)
print(type(alpha))

<class 'numpy.ndarray'>


In [107]:
Mfcc = np.arange(1,11,1)
Mfcc =  np.reshape(Mfcc, (-1, 2))
Mel_spec = np.arange(1,10,2)

df = pd.DataFrame({'mfcc': list(Mfcc), 'mel_spectogram': Mel_spec})

df.mfcc[1], df.mel_spectogram[1]
print(df.mel_spectogram[3])

7
